In [2]:
import pandas as pd
import numpy as np
import jqdatasdk as jq

In [3]:
jq.auth('13918852005','960312Lsc')
# jq.auth('15821912507','912507')

auth success （JQData现有流量增加活动，详情请咨询JQData管理员，微信号：JQData01）


In [6]:
# tradeday = jq.get_trade_days(start_date='2009-01-01', end_date='2019-01-01', count=None)

In [4]:
# 沪深两市自2009年1月1日前上市股票列表
hslslist = list(jq.get_all_securities(types=['stock'], date='2011-01-01').index)

In [5]:
len(hslslist)

2042

In [7]:
# tradedaylen = len(tradeday)

2431

In [5]:
# 沪深两市自2009年1月1日前上市股票列表
# hslist = jq.get_index_stocks('000001.XSHG', date='2009-01-01')
# sslist = jq.get_index_stocks('399106.XSHE', date='2009-01-01')
# hslslist = hslist + sslist

In [20]:
cnt = 0
drop_cnt = 0
stock_num = len(hslslist) # 总股票数-未剔除有停牌
stock_lst = []

# 增量下载才开启，先检验是否已经下载
save_df = pd.read_csv('E:/Stock_Data/bin_data.csv',index_col=0)
code_lst = list(set(list(save_df['code'])))
cnt = int(len(save_df)/8)

for code in hslslist:
    
    # 增量下载才开启，先检验是否已经下载
    if code in code_lst:
        continue

    # 1 利润表
    incomestatement = jq.finance.run_query(
        jq.query(
            jq.finance.STK_INCOME_STATEMENT.code,
            jq.finance.STK_INCOME_STATEMENT.end_date,
            jq.finance.STK_INCOME_STATEMENT.operating_profit,
        ).filter(jq.finance.STK_INCOME_STATEMENT.code==code,
                 jq.finance.STK_INCOME_STATEMENT.end_date>='2009-12-31',
                 jq.finance.STK_INCOME_STATEMENT.end_date<='2018-01-01',
                 jq.finance.STK_INCOME_STATEMENT.source_id=='321003'
                ))
    ## 筛选至仅剩一年一份
    incomestatement['report_type'] = incomestatement['end_date'].apply(lambda x : str(x)[-5:])
    incomestatement = incomestatement[incomestatement['report_type']=='12-31']
    incomestatement = incomestatement.groupby(incomestatement['end_date']).max()[['code','operating_profit']]
    
    # 插入检验，是否存在停牌
    if len(incomestatement.index) != 9:
        cnt += 1
        drop_cnt += 1
        continue
    
    # 2 资产负债表
    balancesheet = jq.finance.run_query(
        jq.query(
            jq.finance.STK_BALANCE_SHEET.code,
            jq.finance.STK_BALANCE_SHEET.end_date,
            jq.finance.STK_BALANCE_SHEET.total_assets,
            jq.finance.STK_BALANCE_SHEET.total_owner_equities,
        ).filter(jq.finance.STK_BALANCE_SHEET.code==code,
                 jq.finance.STK_BALANCE_SHEET.end_date>='2009-12-31',
                 jq.finance.STK_BALANCE_SHEET.end_date<='2018-01-01',
                 jq.finance.STK_BALANCE_SHEET.source_id=='321003'
                ))
    ## 筛选至仅剩一年一份
    balancesheet['report_type'] = balancesheet['end_date'].apply(lambda x : str(x)[-5:])
    balancesheet = balancesheet[balancesheet['report_type']=='12-31']
    balancesheet = balancesheet.groupby(balancesheet['end_date']).max()[['total_assets','total_owner_equities']]
    
    # 插入检验，是否存在停牌
    if len(balancesheet.index) != 9:
        cnt += 1
        drop_cnt += 1
        continue
    
    # 3 估值表
    used_lst = ['market_cap','pe_ratio']
    valuation = pd.DataFrame(index=balancesheet.index,columns=used_lst)
    year_end_lst = [str(2009+i)+'-12-31' for i in range(9)]
    for i in range(9):
        year = str(2009+i)+'-12-31'
        valuation_inf = jq.get_fundamentals(
            jq.query(
                jq.valuation.market_cap,
                jq.valuation.pe_ratio
            ).filter(jq.valuation.code==code),year)      
        if len(valuation_inf) == 0:
            continue
        valuation.iloc[i,] = valuation_inf.iloc[0,]
        
    # 插入检验，确保年报完整
    if len(valuation.dropna()) != 9:
        cnt += 1
        drop_cnt += 1
        continue
     
    # 4 行业
    industry_dct = jq.get_industry(code, date='2019-01-01')
    if 'jq_l1' in industry_dct[code].keys():
        industry_name = industry_dct[code]['jq_l1']['industry_name']
    else:
        industry_name = '其他'
    industry_name_lst = [industry_name for i in range(9)]
    ind_lst = ['industry']
    industry = pd.DataFrame(industry_name_lst,index=balancesheet.index,columns=ind_lst)
    
    # 5 股东类别
#     jq.finance.run_query(jq.query(
#         jq.finance.STK_SHAREHOLDER_TOP10.shareholder_class
#     ).filter(jq.finance.STK_SHAREHOLDER_TOP10.code==code))
        
    # 合并
    bin_df = pd.concat([incomestatement,balancesheet,valuation,industry],axis=1)
    
    # 计算
    ## 投资分组
    bin_df['total_assets_1'] = bin_df['total_assets'].shift(1)
    bin_df['Inv'] = bin_df['total_assets']/bin_df['total_assets_1']-1
    ## 盈利能力分组
    bin_df['OP'] = bin_df['operating_profit']/ bin_df['total_owner_equities']
    ## 账面市值比分组
    bin_df['B_M'] = bin_df['total_owner_equities']/ (bin_df['market_cap']*100000000)
    ## 规模比分组
    bin_df['Size'] = bin_df['market_cap']
    
    # 生成时间列，删选有用组
    bin_df['date'] = bin_df.index
    bin_df = bin_df[['code','date','Size','B_M','OP','Inv','pe_ratio','industry']].iloc[1:,:]
#     bin_df.index = list(range(8))
    
    if cnt == 0:
        save_df = bin_df
    else:
        save_df = pd.concat([save_df,bin_df],axis=0)
    
    cnt += 1
    stock_lst.append(code) # 成功运行的股票记录
    
    print('Complete:',cnt/stock_num*100,'% Total:',cnt-drop_cnt,end=' \r')
    
    if cnt%100 == 0:
        save_df.to_csv('E:/Stock_Data/bin_data.csv',index=True)
        pd.DataFrame(stock_lst).to_csv('E:/Stock_Data/stock_code.csv',index=False)

In [19]:
save_df.to_csv('E:/Stock_Data/bin_data.csv',index=True)
pd.DataFrame(stock_lst).to_csv('E:/Stock_Data/stock_code.csv',index=False)